In [1]:
import os
from openpyxl import load_workbook
import pandas as pd

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')

xls_filename = 'load_and_generation.xlsx'
xls_sheetname = 'Combined Data'
header_col_num = 1 

In [2]:
wb = load_workbook(os.path.join(DATA_DIR, xls_filename), data_only=True)
ws = wb[xls_sheetname]

ws_vals = [row for row in ws.values]
ws_colnames = ws_vals[header_col_num]

load_and_gen_df = pd.DataFrame(data= ws_vals[header_col_num + 1:], columns= ws_colnames)
load_and_gen_df.columns = ['date', 'load_mw', 'gen_wind_mw', 'gen_solar_mw', 'gen_thermal_total_mw', 'gen_thermal_rice_mw', 'gen_thermal_combustion_mw']
load_and_gen_df = load_and_gen_df[['date', 'load_mw', 'gen_wind_mw', 'gen_solar_mw']]
load_and_gen_df.describe()

,load_mw,gen_wind_mw,gen_solar_mw
count,8759.000000,8759.000000,8759.000000
mean,41.792990,5.711954,5.403371
std,5.316065,4.145358,7.175185
min,31.000000,0.000000,-0.007002
25%,37.000000,2.347190,-0.007002
50%,42.800000,4.741340,-0.007002
75%,46.200000,8.400130,11.222235
max,55.800000,15.098400,22.876920


In [15]:
load_and_gen_df['gen_solar_mw'] = load_and_gen_df['gen_solar_mw'].apply(lambda x: x if x > 0 else 0)
load_and_gen_df['gen_thermal_total_mw'] = load_and_gen_df['load_mw'] - load_and_gen_df['gen_wind_mw'] - load_and_gen_df['gen_solar_mw']
load_and_gen_df['gen_thermal_total_kwh'] = load_and_gen_df['gen_thermal_total_mw'] * 1000
load_and_gen_df

,date,load_mw,gen_wind_mw,gen_solar_mw,gen_thermal_total_mw,gen_thermal_total_kwh
0,2017-01-01 01:00:00.000000,35.1,1.67966,0.0,33.42034,33420.34
1,2017-01-01 02:00:00.000000,34.6,1.23327,0.0,33.36673,33366.73
2,2017-01-01 02:59:59.999999,34.2,1.09193,0.0,33.10807,33108.07
3,2017-01-01 03:59:59.999999,33.8,1.46510,0.0,32.33490,32334.90
4,2017-01-01 04:59:59.999999,33.4,1.12029,0.0,32.27971,32279.71
...,...,...,...,...,...,...
8754,2017-12-31 18:59:59.998165,45.5,6.67250,0.0,38.82750,38827.50
8755,2017-12-31 19:59:59.998165,46.5,6.76534,0.0,39.73466,39734.66
8756,2017-12-31 20:59:59.998165,45.0,6.20556,0.0,38.79444,38794.44
8757,2017-12-31 21:59:59.998165,41.7,8.72619,0.0,32.97381,32973.81


In [4]:
'''
values for RICE taken from: https://www.eergy.gov/sites/prod/files/2016/09/f33/CHP-Recip%20Engines.pdf
assumed to be System #5 in paper above, for two reasons:
- System #5 has highest capacity
- System #5 has O&M similar to what is specified in assignment
'''
therm_rice_config = {
    "o_and_m" : .0105, # in cents / kwh
    "nominal_heat_rate_btu" : 9827, # in BTU / kWh
    "fuel_cost" : 2.50, # in $ / gallon
    "thermal_efficiency" : .35,
    "electric_efficiency" : .419,
    "total_efficiency" : .769,
    "column_prepend" : "rice"
}

In [5]:
'''
values for combustion engine taken from: https://www.epa.gov/sites/production/files/2015-07/documents/catalog_of_chp_technologies_section_3._technology_characterization_-_combustion_turbines.pdf
Specifically, Table 3-2. 
Numbers reflect System #1 in paper above, for two reasons:
- System #1 has closest electric heat rate to what is specified in assignment PDF

Annual Tech Baseline - EIA
'''
therm_combustion_config = {
    "o_and_m" : .0046, # in $ / kWh
    "nominal_heat_rate_btu" : 17680, # in BTU / kWh 
    "fuel_cost" : 2.50, # in $ / gallon
    "thermal_efficiency" : .42,
    "electric_efficiency" : .24,
    "total_efficiency" : .657,
    "column_prepend" : "combustion"
}

In [6]:
def standardize_units_of_config(config):
    
    config["nominal_heat_rate_gal"] = config["nominal_heat_rate_btu"] / 139_000 # specified in assignment
    
    return config
    

In [7]:
therm_combustion_config = standardize_units_of_config(therm_combustion_config)
therm_rice_config = standardize_units_of_config(therm_rice_config)

print("--- RICE config:\n{}\n".format(therm_rice_config))
print("--- Combustion config:\n{}\n".format(therm_combustion_config))

--- RICE config:
{'o_and_m': 0.0105, 'nominal_heat_rate_btu': 9827, 'fuel_cost': 2.5, 'thermal_efficiency': 0.35, 'electric_efficiency': 0.419, 'total_efficiency': 0.769, 'column_prepend': 'rice', 'nominal_heat_rate_gal': 0.07069784172661871}

--- Combustion config:
{'o_and_m': 0.0046, 'nominal_heat_rate_btu': 17680, 'fuel_cost': 2.5, 'thermal_efficiency': 0.42, 'electric_efficiency': 0.24, 'total_efficiency': 0.657, 'column_prepend': 'combustion', 'nominal_heat_rate_gal': 0.12719424460431655}



In [16]:
load_and_gen_df

,date,load_mw,gen_wind_mw,gen_solar_mw,gen_thermal_total_mw,gen_thermal_total_kwh
0,2017-01-01 01:00:00.000000,35.1,1.67966,0.0,33.42034,33420.34
1,2017-01-01 02:00:00.000000,34.6,1.23327,0.0,33.36673,33366.73
2,2017-01-01 02:59:59.999999,34.2,1.09193,0.0,33.10807,33108.07
3,2017-01-01 03:59:59.999999,33.8,1.46510,0.0,32.33490,32334.90
4,2017-01-01 04:59:59.999999,33.4,1.12029,0.0,32.27971,32279.71
...,...,...,...,...,...,...
8754,2017-12-31 18:59:59.998165,45.5,6.67250,0.0,38.82750,38827.50
8755,2017-12-31 19:59:59.998165,46.5,6.76534,0.0,39.73466,39734.66
8756,2017-12-31 20:59:59.998165,45.0,6.20556,0.0,38.79444,38794.44
8757,2017-12-31 21:59:59.998165,41.7,8.72619,0.0,32.97381,32973.81


In [8]:
def calc_addtl_cols(df, config):
    
    decorated_df = df.copy()
    efficiency_key = 'total_efficiency'
    
    col_prepend = config['column_prepend']
    o_and_m = config['o_and_m']
    nominal_heat_rate_gal = config['nominal_heat_rate_gal']
    fuel_cost = config['fuel_cost']
    efficiency = config[efficiency_key]
    
    decorated_df['{}_fuel_cost'.format(col_prepend)] = decorated_df['gen_thermal_total_kwh'] * nominal_heat_rate_gal * fuel_cost
    decorated_df['{}_o_and_m_cost'.format(col_prepend)] = decorated_df['gen_thermal_total_kwh'] * o_and_m
    decorated_df['{}_total_cost'.format(col_prepend)] = decorated_df['{}_o_and_m_cost'.format(col_prepend)] + decorated_df['{}_fuel_cost'.format(col_prepend)]
    
    return decorated_df
    

In [9]:
load_and_gen_df_decorated = calc_addtl_cols(load_and_gen_df, therm_rice_config)
load_and_gen_df_decorated = calc_addtl_cols(load_and_gen_df_decorated, therm_combustion_config)

In [17]:
load_and_gen_df_decorated

,date,load_mw,gen_wind_mw,gen_solar_mw,gen_thermal_total_mw,gen_thermal_total_kwh,rice_fuel_cost,rice_o_and_m_cost,rice_total_cost,combustion_fuel_cost,combustion_o_and_m_cost,combustion_total_cost,cheaper_gen
0,2017-01-01 01:00:00.000000,35.1,1.67966,0.0,33.42034,33420.34,5906.864769,350.913570,6257.778339,10627.187252,153.733564,10780.920816,rice
1,2017-01-01 02:00:00.000000,34.6,1.23327,0.0,33.36673,33366.73,5897.389491,350.350665,6247.740156,10610.140043,153.486958,10763.627001,rice
2,2017-01-01 02:59:59.999999,34.2,1.09193,0.0,33.10807,33108.07,5851.672732,347.634735,6199.307467,10527.889885,152.297122,10680.187007,rice
3,2017-01-01 03:59:59.999999,33.8,1.46510,0.0,32.33490,32334.90,5715.019106,339.516450,6054.535556,10282.032950,148.740540,10430.773490,rice
4,2017-01-01 04:59:59.999999,33.4,1.12029,0.0,32.27971,32279.71,5705.264571,338.936955,6044.201526,10264.483324,148.486666,10412.969990,rice
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2017-12-31 18:59:59.998165,45.5,6.67250,0.0,38.82750,38827.50,6862.551124,407.688750,7270.239874,12346.586331,178.606500,12525.192831,rice
8755,2017-12-31 19:59:59.998165,46.5,6.76534,0.0,39.73466,39734.66,7022.886759,417.213930,7440.100689,12635.050158,182.779436,12817.829594,rice
8756,2017-12-31 20:59:59.998165,45.0,6.20556,0.0,38.79444,38794.44,6856.707947,407.341620,7264.049567,12336.073727,178.454424,12514.528151,rice
8757,2017-12-31 21:59:59.998165,41.7,8.72619,0.0,32.97381,32973.81,5827.943001,346.225005,6174.168006,10485.197137,151.679526,10636.876663,rice


In [10]:
load_and_gen_df_decorated.to_csv(os.path.join(DATA_DIR, "bau.csv"), index=False)

In [11]:
load_and_gen_df_decorated["cheaper_gen"] = load_and_gen_df_decorated.apply(lambda x: "rice" if x['rice_total_cost'] < x['combustion_total_cost'] else "combustion", axis=1)

In [12]:
load_and_gen_df_decorated['rice_total_cost'].describe()

count    8759.000000
mean     5743.566714
std      1588.999539
min       276.526577
25%      4651.416241
50%      5801.461366
75%      6882.764316
max      9854.571178
Name: rice_total_cost, dtype: float64

In [13]:
load_and_gen_df_decorated['rice_total_cost'].sum()

50307900.85167071

In [14]:
load_and_gen_df_decorated['gen_thermal_total_kwh'].sum() / 8760

30670.635755239728

In [18]:
50307900.85167071 / 30670.635755239728

1640.2627338129496

~$50M seems like a reasonable cost for 31MW of capacity, according to this site: https://www.power-eng.com/2017/04/18/questions-and-considerations-for-rice-generation-facilities/

This is only a quick, gut check though. Will find more resources to verify sanity of this number